# Grab the linechart and scatterchart data

The data had multiple inputs for some days so we decided to take the running sum of the new_persons_fully_vaccinated and the average of new_confirmed cases. This way we can use distinct to filter out the multiple days without losing integrity of data.

In [1]:
from google.cloud import bigquery
import pandas as pd
# import config
# !export GOOGLE_APPLICATION_CREDENTIALS="C:\Users\Sebeast\Desktop\GT-Data-Analytics-Bootcamp\Project2\coronavirus19-dashboard-04be631d347b.json"
client = bigquery.Client()
QUERY = (
    'SELECT DISTINCT date, country_name, sum(new_persons_fully_vaccinated) OVER (PARTITION BY country_name ORDER BY date) as cum_new_ppl_fully_vaxxed, avg(new_confirmed) OVER (PARTITION BY country_name ORDER BY date) as avg_new_confirmed_cases, avg(new_deceased) OVER (PARTITION BY country_name ORDER BY date) as avg_new_deceased, sum(new_deceased) OVER (PARTITION BY country_name ORDER BY date) as cum_deceased FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE cumulative_persons_fully_vaccinated IS NOT NULL AND new_confirmed IS NOT NULL AND new_deceased IS NOT NULL AND cumulative_deceased IS NOT NULL AND country_name IS NOT NULL ORDER BY date ASC'
    )
query_job = client.query(QUERY)
rows = query_job.result()
date = []
cum_new_ppl_fully_vaxxed = []
avg_new_confirmed_cases = []
avg_new_deceased = []
cum_deceased = []
country_name = []
for row in rows:
    date.append(row.date)
    cum_new_ppl_fully_vaxxed.append(row.cum_new_ppl_fully_vaxxed)
    avg_new_confirmed_cases.append(row.avg_new_confirmed_cases)
    avg_new_deceased.append(row.avg_new_deceased)
    cum_deceased.append(row.cum_deceased)
    country_name.append(row.country_name)
chartdata = pd.DataFrame(cum_new_ppl_fully_vaxxed,date).reset_index().rename(columns={"index":"date",0:"cum_new_ppl_fully_vaxxed"})
chartdata["avg_new_confirmed"] = avg_new_confirmed_cases
chartdata["avg_new_deceased"] = avg_new_deceased
chartdata["cum_deceased"] = cum_deceased
chartdata["country"] = country_name
second_column = chartdata.pop('country')
chartdata.insert(1, 'country', second_column)
chartdata = chartdata.dropna().sort_values(["date","country"], ascending=True)
chartdata

,date,country,cum_new_ppl_fully_vaxxed,avg_new_confirmed,avg_new_deceased,cum_deceased
0,2020-01-20,Brazil,0.0,0.000000,0.000000,0
1,2020-01-21,Brazil,0.0,0.000000,0.000000,0
2,2020-01-28,Brazil,0.0,0.000000,0.000000,0
3,2020-01-30,Brazil,0.0,0.000000,0.000000,0
4,2020-02-05,Brazil,0.0,0.000000,0.000000,0
...,...,...,...,...,...,...
6189,2021-04-28,Ukraine,9.0,12778.729730,342.729730,12681
6184,2021-04-28,United Kingdom,35184374.0,2842.836985,97.638037,111405
6190,2021-04-28,United States of America,188238806.0,1632.324022,30.157860,457736
6194,2021-04-29,Australia,132968.0,2.287425,0.000000,0


In [2]:
# Create Global Data
globalchartdata = chartdata[['date','cum_new_ppl_fully_vaxxed','avg_new_confirmed','avg_new_deceased','cum_deceased']]
globalchartdata = chartdata.groupby("date").sum().reset_index()
globalchartdata['country'] = 'Global'
globalchartdata
chartdata = pd.concat([chartdata,globalchartdata])

In [3]:
chartdata = chartdata.dropna().sort_values(["date","country"], ascending=True)

In [4]:
chartdata

,date,country,cum_new_ppl_fully_vaxxed,avg_new_confirmed,avg_new_deceased,cum_deceased
0,2020-01-20,Brazil,0.0,0.000000,0.000000,0
0,2020-01-20,Global,0.0,0.000000,0.000000,0
1,2020-01-21,Brazil,0.0,0.000000,0.000000,0
1,2020-01-21,Global,0.0,0.000000,0.000000,0
2,2020-01-28,Brazil,0.0,0.000000,0.000000,0
...,...,...,...,...,...,...
6184,2021-04-28,United Kingdom,35184374.0,2842.836985,97.638037,111405
6190,2021-04-28,United States of America,188238806.0,1632.324022,30.157860,457736
6194,2021-04-29,Australia,132968.0,2.287425,0.000000,0
418,2021-04-29,Global,11237155.0,1591.551389,39.636740,95802


In [5]:
allGroup = ['Global','Albania', 'Andorra', 'Argentina', 'Aruba', 'Australia', 'Austria',
                    'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium',
                    'Bermuda', 'Bolivia', 'Brazil', 'Bulgaria', 'Cambodia', 'Canada',
                    'Cayman Islands', 'Chile', 'Colombia', 'Costa Rica', 'Croatia',
                    'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Dominica',
                    'Dominican Republic', 'Ecuador', 'El Salvador',
                    'Equatorial Guinea', 'Estonia', 'Falkland Islands',
                    'Faroe Islands', 'Finland', 'France', 'Germany', 'Gibraltar',
                    'Greece', 'Greenland', 'Guatemala', 'Guernsey', 'Guinea',
                    'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran',
                    'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Japan', 'Jersey',
                    'Jordan', 'Kazakhstan', 'Kuwait', 'Laos', 'Latvia', 'Lebanon',
                    'Liechtenstein', 'Lithuania', 'Luxembourg', 'Malaysia', 'Maldives',
                    'Malta', 'Marshall Islands', 'Mexico', 'Moldova', 'Monaco',
                    'Montenegro', 'Montserrat', 'Morocco', 'Netherlands',
                    'New Zealand', 'Norway', 'Oman', 'Palau', 'Palestine', 'Panama',
                    'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Romania',
                    'Russia', 'Saint Helena', 'San Marino', 'Serbia', 'Seychelles',
                    'Sierra Leone', 'Singapore', 'Slovakia', 'Slovenia',
                    'South Africa', 'South Korea', 'Spain', 'Sweden', 'Switzerland',
                    'Thailand', 'Tunisia', 'Turkey', 'Ukraine', 'United Arab Emirates',
                    'United Kingdom', 'United States of America', 'Uruguay',
                    'Zimbabwe']
df_dict = {}
for country in allGroup:
    df_dict[country] = chartdata[chartdata['country'] == country]

In [6]:
global_scatterchart = df_dict["Global"]

In [7]:
import numpy as np
population_viz = []
for key in df_dict:
    population_viz.append(df_dict[key].tail(1))
population_viz = pd.DataFrame(np.concatenate(population_viz))
population_viz = population_viz.rename(columns={0:"date",1:"country",2:"cum_new_ppl_fully_vaxxed",3:"avg_new_confirmed",
                               4:"avg_new_deceased",5:"cum_deceased"})

In [8]:
population_viz = population_viz.drop(population_viz.index[0])
population_viz

,date,country,cum_new_ppl_fully_vaxxed,avg_new_confirmed,avg_new_deceased,cum_deceased
1,2021-02-22,Albania,654,1070.8,14.4,72
2,2021-04-19,Andorra,3509,46,0,0
3,2021-04-28,Argentina,3.8057e+07,1002.9,13.0549,25692
4,2021-04-26,Aruba,9950,31.7647,0.352941,6
5,2021-04-29,Australia,132968,2.28743,0,0
...,...,...,...,...,...,...
106,2021-04-20,United Arab Emirates,3.58652e+06,2479,8,24
107,2021-04-28,United Kingdom,3.51844e+07,2842.84,97.638,111405
108,2021-04-28,United States of America,1.88239e+08,1632.32,30.1579,457736
109,2021-04-27,Uruguay,553820,3054.78,49.2812,1577


In [9]:
import pandas as pd
allGroup2 = ['Albania', 'Andorra', 'Argentina', 'Aruba', 'Australia', 'Austria',
                    'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium',
                    'Bermuda', 'Bolivia', 'Brazil', 'Bulgaria', 'Cambodia', 'Canada',
                    'Cayman Islands', 'Chile', 'Colombia', 'Costa Rica', 'Croatia',
                    'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Dominica',
                    'Dominican Republic', 'Ecuador', 'El Salvador',
                    'Equatorial Guinea', 'Estonia', 'Falkland Islands',
                    'Faroe Islands', 'Finland', 'France', 'Germany', 'Gibraltar',
                    'Greece', 'Greenland', 'Guatemala', 'Guernsey', 'Guinea',
                    'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran',
                    'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Japan', 'Jersey',
                    'Jordan', 'Kazakhstan', 'Kuwait', 'Laos', 'Latvia', 'Lebanon',
                    'Liechtenstein', 'Lithuania', 'Luxembourg', 'Malaysia', 'Maldives',
                    'Malta', 'Marshall Islands', 'Mexico', 'Moldova', 'Monaco',
                    'Montenegro', 'Montserrat', 'Morocco', 'Netherlands',
                    'New Zealand', 'Norway', 'Oman', 'Palau', 'Palestine', 'Panama',
                    'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Romania',
                    'Russia', 'Saint Helena', 'San Marino', 'Serbia', 'Seychelles',
                    'Sierra Leone', 'Singapore', 'Slovakia', 'Slovenia',
                    'South Africa', 'South Korea', 'Spain', 'Sweden', 'Switzerland',
                    'Thailand', 'Tunisia', 'Turkey', 'Ukraine', 'United Arab Emirates',
                    'United Kingdom', 'United States', 'Uruguay',
                    'Zimbabwe']
url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'
tables = pd.read_html(url)
pop_data = tables[0]
pop_data = pop_data[['Country(or dependent territory)','Population']].rename(columns={'Country(or dependent territory)':'country',
                                                                                      'Population':'population'})
pop_data['country'] = pop_data['country'].str.split('[').str[0]
pop_data = pop_data[pop_data['country'].isin(allGroup2)].reset_index()
pop_data.loc[(pop_data.country == 'United States'),'country']='United States of America'
pop_data

,index,country,population
0,1,India,1376318536
1,2,United States of America,331449281
2,3,Indonesia,271350000
3,5,Brazil,213066609
4,7,Bangladesh,170577312
...,...,...,...
92,210,Marshall Islands,54516
93,216,Liechtenstein,39062
94,217,Monaco,38350
95,220,San Marino,33604


In [10]:
merged_inner = pd.merge(left=population_viz, right=pop_data, left_on='country', right_on='country')
merged_inner

,date,country,cum_new_ppl_fully_vaxxed,avg_new_confirmed,avg_new_deceased,cum_deceased,index,population
0,2021-02-22,Albania,654,1070.8,14.4,72,137,2845955
1,2021-04-19,Andorra,3509,46,0,0,202,78015
2,2021-04-28,Argentina,3.8057e+07,1002.9,13.0549,25692,30,45808747
3,2021-04-29,Australia,132968,2.28743,0,0,52,25788571
4,2021-04-28,Austria,1.00063e+06,572.959,8.85338,4710,98,8933346
...,...,...,...,...,...,...,...,...
92,2021-04-20,United Arab Emirates,3.58652e+06,2479,8,24,92,9503738
93,2021-04-28,United Kingdom,3.51844e+07,2842.84,97.638,111405,20,66796807
94,2021-04-28,United States of America,1.88239e+08,1632.32,30.1579,457736,2,331449281
95,2021-04-27,Uruguay,553820,3054.78,49.2812,1577,132,3554915


In [11]:
merged_inner["perc_ppl_fully_vaxxed"] = str(merged_inner["cum_new_ppl_fully_vaxxed"]/merged_inner["population"]*100)
merged_inner = merged_inner[['country','perc_ppl_fully_vaxxed']]
merged_inner

,country,perc_ppl_fully_vaxxed
0,Albania,0 0.02298\n1 4.49785\n2 83.0781...
1,Andorra,0 0.02298\n1 4.49785\n2 83.0781...
2,Argentina,0 0.02298\n1 4.49785\n2 83.0781...
3,Australia,0 0.02298\n1 4.49785\n2 83.0781...
4,Austria,0 0.02298\n1 4.49785\n2 83.0781...
...,...,...
92,United Arab Emirates,0 0.02298\n1 4.49785\n2 83.0781...
93,United Kingdom,0 0.02298\n1 4.49785\n2 83.0781...
94,United States of America,0 0.02298\n1 4.49785\n2 83.0781...
95,Uruguay,0 0.02298\n1 4.49785\n2 83.0781...


In [14]:
merged_inner = merged_inner.sort_values(['perc_ppl_fully_vaxxed'], ascending=False).reset_index(drop=True)
top_ten = merged_inner[3:12]

,country,perc_ppl_fully_vaxxed
3,Seychelles,0.578428
4,United States of America,0.567926
5,United Kingdom,0.526737
6,Palau,0.446177
7,United Arab Emirates,0.37738
8,Chile,0.328318
9,San Marino,0.256874
10,Maldives,0.23723
11,Bahrain,0.220056


In [17]:
globalchartdata = chartdata[['date','cum_new_ppl_fully_vaxxed','avg_new_confirmed','avg_new_deceased','cum_deceased']]
globalchartdata = chartdata.groupby("date").sum()
globalchartdata

,cum_new_ppl_fully_vaxxed,avg_new_confirmed,avg_new_deceased,cum_deceased
date,,,,
2020-01-20,0.0,0.000000,0.000000,0
2020-01-21,0.0,0.000000,0.000000,0
2020-01-28,0.0,0.000000,0.000000,0
2020-01-30,0.0,0.000000,0.000000,0
2020-02-05,0.0,0.000000,0.000000,0
...,...,...,...,...
2021-04-19,443930491.0,273241.783028,4941.198394,1700488
2021-04-20,439593172.0,209985.182628,4595.564777,1689962
2021-04-21,459726302.0,255012.296480,4530.839653,1730250


In [3]:
client = bigquery.Client()
QUERY = (
    'SELECT DISTINCT date, sum(new_persons_fully_vaccinated) OVER (ORDER BY date) as cum_new_ppl_fully_vaxxed, avg(new_confirmed) OVER (ORDER BY date) as avg_new_confirmed_cases FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE country_code = "BR" AND cumulative_persons_fully_vaccinated IS NOT NULL AND cumulative_persons_fully_vaccinated != 0 AND new_confirmed IS NOT NULL ORDER BY date ASC'
    )
query_job = client.query(QUERY)
rows = query_job.result()
date = []
cum_new_ppl_fully_vaxxed = []
avg_new_confirmed_cases = []
for row in rows:
    date.append(row.date)
    cum_new_ppl_fully_vaxxed.append(row.cum_new_ppl_fully_vaxxed)
    avg_new_confirmed_cases.append(row.avg_new_confirmed_cases)
BR_linechart = pd.DataFrame(cum_new_ppl_fully_vaxxed,date).reset_index().rename(columns={"index":"date",0:"cum_new_ppl_fully_vaxxed"})
BR_linechart["avg_new_confirmed"] = avg_new_confirmed_cases
BR_linechart["Country"] = "Brazil"
BR_linechart

,date,cum_new_ppl_fully_vaxxed,avg_new_confirmed,Country
0,2021-01-03,9,5794.000000,Brazil
1,2021-01-04,9,9347.000000,Brazil
2,2021-01-05,9,15699.666667,Brazil
3,2021-01-06,9,22518.285714,Brazil
4,2021-01-08,9,23307.444444,Brazil
...,...,...,...,...
103,2021-04-17,28863488,47.945673,Brazil
104,2021-04-18,29039758,48.214684,Brazil
105,2021-04-19,30232538,48.405092,Brazil
106,2021-04-20,31277298,48.841850,Brazil


In [4]:
BR_linechart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      108 non-null    object 
 1   cum_new_ppl_fully_vaxxed  108 non-null    int64  
 2   avg_new_confirmed         108 non-null    float64
 3   Country                   108 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 3.5+ KB


In [5]:
client = bigquery.Client()
QUERY = (
    'SELECT DISTINCT date, sum(new_persons_fully_vaccinated) OVER (ORDER BY date) as cum_new_ppl_fully_vaxxed, avg(new_confirmed) OVER (ORDER BY date) as avg_new_confirmed_cases FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE country_code = "IN" AND cumulative_persons_fully_vaccinated IS NOT NULL AND new_confirmed IS NOT NULL ORDER BY date ASC'
    )
query_job = client.query(QUERY)
rows = query_job.result()
date = []
cum_new_ppl_fully_vaxxed = []
avg_new_confirmed_cases = []
for row in rows:
    date.append(row.date)
    cum_new_ppl_fully_vaxxed.append(row.cum_new_ppl_fully_vaxxed)
    avg_new_confirmed_cases.append(row.avg_new_confirmed_cases)
IN_linechart = pd.DataFrame(cum_new_ppl_fully_vaxxed,date).reset_index().rename(columns={"index":"date",0:"cum_new_ppl_fully_vaxxed"})
IN_linechart["avg_new_confirmed"] = avg_new_confirmed_cases
IN_linechart["Country"] = "India"
IN_linechart

,date,cum_new_ppl_fully_vaxxed,avg_new_confirmed,Country
0,2021-02-13,NaN,11649.000000,India
1,2021-02-15,90450.0,11629.500000,India
2,2021-02-16,208671.0,12046.666667,India
3,2021-02-17,334448.0,12333.250000,India
4,2021-02-18,457264.0,12665.200000,India
...,...,...,...,...
60,2021-04-15,14866586.0,59222.098361,India
61,2021-04-16,15521489.0,62484.645161,India
62,2021-04-17,16183846.0,65839.015873,India
63,2021-04-18,16473128.0,68859.812500,India


In [6]:
# Drop first row containing the single null value
IN_linechart = IN_linechart.dropna()
IN_linechart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 1 to 64
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      64 non-null     object 
 1   cum_new_ppl_fully_vaxxed  64 non-null     float64
 2   avg_new_confirmed         64 non-null     float64
 3   Country                   64 non-null     object 
dtypes: float64(2), object(2)
memory usage: 2.5+ KB


In [7]:
client = bigquery.Client()
QUERY = (
    'SELECT DISTINCT date, sum(new_persons_fully_vaccinated) OVER (ORDER BY date) as cum_new_ppl_fully_vaxxed, avg(new_confirmed) OVER (ORDER BY date) as avg_new_confirmed_cases FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE country_code = "IT" AND cumulative_persons_fully_vaccinated IS NOT NULL AND cumulative_persons_fully_vaccinated != 0 AND new_confirmed IS NOT NULL ORDER BY date ASC'
    )
query_job = client.query(QUERY)
rows = query_job.result()
date = []
cum_new_ppl_fully_vaxxed = []
avg_new_confirmed_cases = []
for row in rows:
    date.append(row.date)
    cum_new_ppl_fully_vaxxed.append(row.cum_new_ppl_fully_vaxxed)
    avg_new_confirmed_cases.append(row.avg_new_confirmed_cases)
IT_linechart = pd.DataFrame(cum_new_ppl_fully_vaxxed,date).reset_index().rename(columns={"index":"date",0:"cum_new_ppl_fully_vaxxed"})
IT_linechart["avg_new_confirmed"] = avg_new_confirmed_cases
IT_linechart["Country"] = "Italy"
IT_linechart

,date,cum_new_ppl_fully_vaxxed,avg_new_confirmed,Country
0,2021-01-17,5807,1491.437500,Italy
1,2021-01-18,15605,1179.457143,Italy
2,2021-01-19,18869,1141.888889,Italy
3,2021-01-20,27659,1206.452055,Italy
4,2021-01-21,65881,1255.217391,Italy
...,...,...,...,...
91,2021-04-18,8887382,1613.464835,Italy
92,2021-04-19,9060098,1605.549457,Italy
93,2021-04-20,9250365,1601.192473,Italy
94,2021-04-21,9471177,1598.761702,Italy


In [8]:
IT_linechart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      96 non-null     object 
 1   cum_new_ppl_fully_vaxxed  96 non-null     int64  
 2   avg_new_confirmed         96 non-null     float64
 3   Country                   96 non-null     object 
dtypes: float64(1), int64(1), object(2)
memory usage: 3.1+ KB


In [9]:
client = bigquery.Client()
QUERY = (
    'SELECT DISTINCT date, sum(new_persons_fully_vaccinated) OVER (ORDER BY date) as cum_new_ppl_fully_vaxxed, avg(new_confirmed) OVER (ORDER BY date) as avg_new_confirmed_cases FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE country_code = "ID" AND cumulative_persons_fully_vaccinated IS NOT NULL AND cumulative_persons_fully_vaccinated != 0 AND new_confirmed IS NOT NULL ORDER BY date ASC'
    )
query_job = client.query(QUERY)
rows = query_job.result()
date = []
cum_new_ppl_fully_vaxxed = []
avg_new_confirmed_cases = []
for row in rows:
    date.append(row.date)
    cum_new_ppl_fully_vaxxed.append(row.cum_new_ppl_fully_vaxxed)
    avg_new_confirmed_cases.append(row.avg_new_confirmed_cases)
ID_linechart = pd.DataFrame(cum_new_ppl_fully_vaxxed,date).reset_index().rename(columns={"index":"date",0:"cum_new_ppl_fully_vaxxed"})
ID_linechart["avg_new_confirmed"] = avg_new_confirmed_cases
ID_linechart["Country"] = "Indonesia"
ID_linechart

,date,cum_new_ppl_fully_vaxxed,avg_new_confirmed,Country
0,2021-01-28,NaN,13695.000000,Indonesia
1,2021-01-29,5819.0,13748.500000,Indonesia
2,2021-01-30,15342.0,14005.000000,Indonesia
3,2021-01-31,17080.0,13504.000000,Indonesia
4,2021-02-01,29938.0,13002.000000,Indonesia
...,...,...,...,...
60,2021-04-15,5711992.0,7489.803279,Indonesia
61,2021-04-17,5885322.0,7450.306452,Indonesia
62,2021-04-19,6047144.0,7410.650794,Indonesia
63,2021-04-20,6153280.0,7381.562500,Indonesia


In [10]:
# Drop first row containing the single null value
ID_linechart = ID_linechart.dropna()
ID_linechart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 1 to 64
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      64 non-null     object 
 1   cum_new_ppl_fully_vaxxed  64 non-null     float64
 2   avg_new_confirmed         64 non-null     float64
 3   Country                   64 non-null     object 
dtypes: float64(2), object(2)
memory usage: 2.5+ KB


In [11]:
linechart = pd.concat([US_linechart,BR_linechart,IN_linechart,IT_linechart,ID_linechart]).reset_index(drop=True)
linechart

,date,cum_new_ppl_fully_vaxxed,avg_new_confirmed,Country
0,2020-12-13,1355.0,180420.000000,United States of America
1,2020-12-14,1360.0,192350.500000,United States of America
2,2020-12-15,1386.0,195389.666667,United States of America
3,2020-12-16,1589.0,205493.500000,United States of America
4,2020-12-17,2097.0,210377.800000,United States of America
...,...,...,...,...
457,2021-04-15,5711992.0,7489.803279,Indonesia
458,2021-04-17,5885322.0,7450.306452,Indonesia
459,2021-04-19,6047144.0,7410.650794,Indonesia
460,2021-04-20,6153280.0,7381.562500,Indonesia


# Grab the heatmap data

Data is from the start of 2021 to the current day.

In [15]:
client = bigquery.Client()
QUERY = (
         'SELECT date, cumulative_persons_fully_vaccinated, latitude, longitude, country_name FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE date >= "2021-01-01" and date <= current_date() AND cumulative_persons_fully_vaccinated IS NOT NULL AND cumulative_persons_fully_vaccinated != 0 ORDER BY date DESC'
        )
query_job = client.query(QUERY)
rows = query_job.result()
date = []
cumulative_persons_fully_vaccinated = []
latitude = []
longitude = []
country_name = []
for row in rows:
    date.append(row.date)
    cumulative_persons_fully_vaccinated.append(row.cumulative_persons_fully_vaccinated)
    latitude.append(row.latitude)
    longitude.append(row.longitude)
    country_name.append(row.country_name)
heatmap = pd.DataFrame(cumulative_persons_fully_vaccinated,date).reset_index().rename(columns={"index":"date",0:"cumulative_persons_fully_vaccinated"})
heatmap["latitude"] = latitude
heatmap["longitude"] = longitude
heatmap["country_name"] = country_name
heatmap

,date,cumulative_persons_fully_vaccinated,latitude,longitude,country_name
0,2021-04-22,8059,-26.183333,-58.175000,Argentina
1,2021-04-22,106105,42.216667,13.833333,Italy
2,2021-04-22,178088,38.895000,-77.036667,United States of America
3,2021-04-22,451914,21.307500,-157.857500,United States of America
4,2021-04-22,936258,42.000000,-93.000000,United States of America
...,...,...,...,...,...
375896,2021-01-02,3,47.164800,8.548000,Switzerland
375897,2021-01-01,3,47.164800,8.548000,Switzerland
375898,2021-01-01,745,51.000000,10.000000,Germany
375899,2021-01-01,1,47.378400,9.312800,Switzerland


In [16]:
heatmap = heatmap.dropna()
linechart = heatmap.linechart()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374671 entries, 0 to 375900
Data columns (total 5 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   date                                 374671 non-null  object 
 1   cumulative_persons_fully_vaccinated  374671 non-null  int64  
 2   latitude                             374671 non-null  float64
 3   longitude                            374671 non-null  float64
 4   country_name                         374671 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 17.2+ MB


In [17]:
heatmap.to_csv('heatmap.csv', index=False)
linechart.to_csv('linechart.csv', index=False)

In [19]:
heatmap = pd.read_csv('heatmap.csv')
linechart = pd.read_csv('linechart.csv')

In [ ]:
# Insert DF into mongoDB
from pymongo import MongoClient
import pandas as pd

client = MongoClient('mongodb://localhost:27017')
db = client.Coronavirus19_Dashboard
collection = db.linechart
data = linechart.to_dict(orient='records')

db.linechartandscatterchart.insert_many(data)

In [20]:
# Insert DF into mongoDB
from pymongo import MongoClient
import pandas as pd

client = MongoClient('mongodb://localhost:27017')
db = client.Coronavirus19_Dashboard
collection = db.heatmap
data = heatmap.to_dict(orient='records')

db.heatmap.insert_many(data)